In [ ]:
features = ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", 
                       "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "snow_depth", 
                       "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", 
                       "cloud_cover_mid", "cloud_cover_high", "visibility", "evapotranspiration", 
                       "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_80m", 
                       "wind_speed_120m", "wind_speed_180m", "wind_direction_10m", "wind_direction_80m", 
                       "wind_direction_120m", "wind_direction_180m", "wind_gusts_10m", "temperature_120m", 
                       "temperature_80m", "temperature_180m", "soil_temperature_0cm", "soil_temperature_6cm", 
                       "soil_temperature_18cm", "soil_temperature_54cm", "soil_moisture_0_to_1cm", 
                       "soil_moisture_1_to_3cm", "soil_moisture_3_to_9cm", "soil_moisture_9_to_27cm", 
                       "soil_moisture_27_to_81cm"]

len(features)

42

## Import Libaries

In [ ]:
# import all necessary libraries
import datetime
from utils.Copernicus import AdvancedCopernicus
import pandas as pd
import numpy as np
import warnings
import folium
from tqdm import tqdm
from pathlib import Path
from utils.OpenMeteoWeather import OpenMeteoWeatherHistory

from datetime import datetime
# Ignore SettingWithCopyWarning:
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)



# Display all columns
pd.options.display.max_columns = None
#pd.options.display.max_rows = None


Api key: e6RDWgdKHpmEfySx
 <class 'str'>


In [2]:
START_DATE = '2022-12-03'
END_DATE = '2025-03-22'
OUTPUT_FILENAME = 'output.nc'

BBOX= {
    
    # "minimum_longitude":9.10,
    # "maximum_longitude":30.20,
    # "minimum_latitude":53.10,
    # "maximum_latitude":65.00,

    "minimum_longitude":9.1,
    "maximum_longitude":23.0,
    "minimum_latitude":53.5,
    "maximum_latitude":60.0,

    # "minimum_longitude":9.85083510071235,
    # "maximum_longitude":10.926709174713364,
    # "minimum_latitude":54.25206332481298,
    # "maximum_latitude":54.97306793985031,

    "target_min_lon":10.156,
    "target_max_lon":10.170,
    "target_min_lat":54.354,
    "target_max_lat":54.365
    }

In [3]:
# Download Data
minimum_latitude = BBOX['minimum_latitude']
maximum_latitude = BBOX['maximum_latitude']
minimum_longitude = BBOX['minimum_longitude']
maximum_longitude = BBOX['maximum_longitude']

copernicus = AdvancedCopernicus()
points = 11
latitudes = np.linspace(minimum_latitude, maximum_latitude, points)
longitudes = np.linspace(minimum_longitude, maximum_longitude, points)



# zip latitudes and longitudes
coordinates = set([(float(lat), float(lon)) for lat in latitudes for lon in longitudes])
print(len(coordinates))

mean_lat = np.mean(latitudes)
mean_lon = np.mean(longitudes)
# plot the coordinates to folium
folium_map = folium.Map(location=[mean_lat, mean_lon], zoom_start=5)
folium.Polygon(locations=[[minimum_latitude, minimum_longitude], [minimum_latitude, maximum_longitude], [maximum_latitude, maximum_longitude], [maximum_latitude, minimum_longitude], [minimum_latitude, minimum_longitude]], color='orange').add_to(folium_map)
for lat, lon in coordinates:
    # plot the coordinates with dots
    

    #folium.Circle(location=[lat, lon], radius=30, color='red').add_to(folium_map)
    folium.CircleMarker(location=[lat, lon], radius=4, color='red').add_to(folium_map)
    #folium.Marker(location=[lat, lon], popup=f'lat: {lat}, lon: {lon}').add_to(folium_map)
folium_map

121


In [4]:
def process_df(df:pd.DataFrame)->pd.DataFrame:
    # df = df.dropna(axis=1, how="all")
    # df = df.dropna(axis=0, how="any")
    # drop cols with lot of missing values
    #df = df.drop(columns=['depth'], axis=1)
    df = df[["time"] + [col for col in df.columns if col != "time"]]
    float_cols = df.select_dtypes(include=["float"]).columns
    df[float_cols] = df[float_cols].astype(np.float32)
    df["time"] = pd.to_datetime(df["time"]).dt.tz_localize(None).dt.round("h")
    
    #df = df.groupby(by=['time', 'latitude', 'longitude']).mean().reset_index()
    df = df.reset_index(drop=True)

    return df

### Weather Data

In [ ]:
latitudes = [lat for lat, lon in coordinates]
longitudes = [lon for lat, lon in coordinates]

    
print(len(latitudes), latitudes)
print(len(longitudes), longitudes)
end_date_dt = datetime.strptime(END_DATE, "%Y-%m-%d")  # angepasstes Format
start_date_dt = datetime.strptime(START_DATE, "%Y-%m-%d")  # angepasstes Format

# Optional: Wenn Sie Zeit hinzufügen möchten, können Sie dies manuell tun
# start_date_dt = start_date_dt.replace(hour=0, minute=0, second=0)
# end_date_dt = end_date_dt.replace(hour=23, minute=59, second=59)

end_str = end_date_dt.strftime("%Y-%m-%d")
start_str = start_date_dt.strftime("%Y-%m-%d")




# Define the chunk size (you can adjust this value based on the API limitations)
chunk_size = 100 #  set to no sep over the API's limit

# Split latitudes and longitudes into smaller chunks
lat_chunks = [latitudes[i:i + chunk_size] for i in range(0, len(latitudes), chunk_size)]
lon_chunks = [longitudes[i:i + chunk_size] for i in range(0, len(longitudes), chunk_size)]

print(len(lat_chunks), lat_chunks)
print(len(lon_chunks), lon_chunks)

121 [54.15, 53.5, 54.8, 60.0, 55.45, 53.5, 59.35, 54.15, 58.05, 53.5, 57.4, 56.1, 58.7, 54.8, 56.1, 60.0, 58.7, 54.8, 56.75, 58.7, 57.4, 56.75, 55.45, 57.4, 58.7, 59.35, 56.1, 58.05, 60.0, 58.7, 60.0, 55.45, 56.75, 59.35, 54.15, 60.0, 58.05, 53.5, 54.15, 56.1, 56.75, 58.05, 54.8, 59.35, 56.1, 54.8, 55.45, 59.35, 57.4, 54.15, 54.8, 57.4, 53.5, 60.0, 55.45, 54.15, 54.8, 57.4, 60.0, 53.5, 53.5, 54.15, 56.1, 58.05, 57.4, 53.5, 56.1, 58.05, 60.0, 56.1, 58.7, 58.7, 58.05, 55.45, 56.1, 56.75, 58.7, 54.15, 55.45, 53.5, 56.75, 54.15, 55.45, 59.35, 58.05, 53.5, 59.35, 58.05, 54.15, 55.45, 54.15, 54.8, 58.7, 53.5, 58.7, 54.8, 57.4, 56.75, 57.4, 60.0, 60.0, 56.75, 58.05, 59.35, 58.7, 60.0, 59.35, 56.1, 58.05, 56.75, 56.1, 54.8, 59.35, 54.8, 56.75, 57.4, 56.75, 55.45, 57.4, 55.45, 59.35]
121 [18.83, 16.05, 10.49, 14.66, 23.0, 18.83, 23.0, 20.22, 11.879999999999999, 20.22, 10.49, 14.66, 14.66, 21.61, 16.05, 10.49, 16.05, 23.0, 11.879999999999999, 18.83, 21.61, 13.27, 11.879999999999999, 23.0, 20.22,

In [6]:
import time 
chunked_frames = []
# Loop over each chunk and make the API request
for lat_chunk, lon_chunk in tqdm(zip(lat_chunks, lon_chunks), total=len(lat_chunks)):
    try:
        weather_data = OpenMeteoWeatherHistory(lat_chunk, lon_chunk, start_str, end_str)
        df_weather = weather_data.process_hourly_data()
        chunked_frames.append(df_weather)
    except Exception as e:
        print(e)
        print('\nWait 1 hour for the next request\n')
        time.sleep(3605) 
        weather_data = OpenMeteoWeatherHistory(lat_chunk, lon_chunk, start_str, end_str)
        df_weather = weather_data.process_hourly_data()
        chunked_frames.append(df_weather)
        
        
        



  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [13:12<13:12, 792.17s/it]

{'error': True, 'reason': 'Daily API request limit exceeded. Please try again tomorrow.'}

Wait 1 hour for the next request



 50%|█████     | 1/2 [1:13:17<1:13:17, 4397.42s/it]


OpenMeteoRequestsError: {'reason': 'Daily API request limit exceeded. Please try again tomorrow.', 'error': True}

In [ ]:
for frame in chunked_frames:
    display(frame.head())
    print(frame.info())
    break

,time,latitude,longitude,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_80m,wind_speed_120m,wind_speed_180m,wind_direction_10m,wind_direction_80m,wind_direction_120m,wind_direction_180m,wind_gusts_10m,temperature_120m,temperature_80m,temperature_180m,soil_temperature_0cm,soil_temperature_6cm,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_to_1cm,soil_moisture_1_to_3cm,soil_moisture_3_to_9cm,soil_moisture_9_to_27cm,soil_moisture_27_to_81cm
0,2022-12-03 00:00:00+00:00,56.766254,9.056604,1.8285,91.745644,0.6285,-2.225537,NaN,0.0,0.0,0.0,0.0,NaN,1.0,1033.500000,1030.295898,46.0,100.0,92.0,0.0,NaN,NaN,0.000000,0.057718,13.905509,NaN,NaN,NaN,68.749428,NaN,NaN,NaN,28.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-12-03 01:00:00+00:00,56.766254,9.056604,2.1785,89.483276,0.6285,-1.960651,NaN,0.0,0.0,0.0,0.0,NaN,3.0,1033.300049,1030.100708,98.0,22.0,98.0,0.0,NaN,NaN,0.000930,0.075397,14.489720,NaN,NaN,NaN,63.435013,NaN,NaN,NaN,23.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-12-03 02:00:00+00:00,56.766254,9.056604,2.4285,84.478004,0.0785,-2.108335,NaN,0.0,0.0,0.0,0.0,NaN,3.0,1032.900024,1029.704834,93.0,1.0,93.0,0.0,NaN,NaN,0.008470,0.113278,16.622490,NaN,NaN,NaN,72.349792,NaN,NaN,NaN,27.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-03 03:00:00+00:00,56.766254,9.056604,1.8785,86.588997,-0.1215,-2.495600,NaN,0.0,0.0,0.0,0.0,NaN,3.0,1032.699951,1029.498901,85.0,85.0,5.0,0.0,NaN,NaN,0.004688,0.094109,15.294703,NaN,NaN,NaN,63.435013,NaN,NaN,NaN,27.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-12-03 04:00:00+00:00,56.766254,9.056604,2.0285,86.290878,-0.0215,-2.476788,NaN,0.0,0.0,0.0,0.0,NaN,3.0,1032.500000,1029.301270,97.0,97.0,0.0,0.0,NaN,NaN,0.005886,0.097237,16.299694,NaN,NaN,NaN,59.470375,NaN,NaN,NaN,26.639999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Index: 504600 entries, 0 to 20183
Data columns (total 45 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   time                        504600 non-null  datetime64[ns, UTC]
 1   latitude                    504600 non-null  float64            
 2   longitude                   504600 non-null  float64            
 3   temperature_2m              504600 non-null  float32            
 4   relative_humidity_2m        504600 non-null  float32            
 5   dew_point_2m                504600 non-null  float32            
 6   apparent_temperature        504600 non-null  float32            
 7   precipitation_probability   0 non-null       float32            
 8   precipitation               504600 non-null  float32            
 9   rain                        504600 non-null  float32            
 10  showers                     504600 non-null  float

In [ ]:

# Verwende pd.concat, um alle DataFrames zu einem einzigen zu verbinden
df_weather = pd.concat(chunked_frames, ignore_index=True)




# Zeige das zusammengeführte DataFrame
display(df_weather)
print(df_weather.info(verbose=True))
print(df_weather.shape)

,time,latitude,longitude,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,evapotranspiration,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_80m,wind_speed_120m,wind_speed_180m,wind_direction_10m,wind_direction_80m,wind_direction_120m,wind_direction_180m,wind_gusts_10m,temperature_120m,temperature_80m,temperature_180m,soil_temperature_0cm,soil_temperature_6cm,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_to_1cm,soil_moisture_1_to_3cm,soil_moisture_3_to_9cm,soil_moisture_9_to_27cm,soil_moisture_27_to_81cm
0,2022-12-03 00:00:00+00:00,56.766254,9.056604,1.8285,91.745644,0.6285,-2.225537,NaN,0.0,0.0,0.0,0.0,NaN,1.0,1033.500000,1030.295898,46.0,100.0,92.0,0.0,NaN,NaN,0.000000,0.057718,13.905509,NaN,NaN,NaN,68.749428,NaN,NaN,NaN,28.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-12-03 01:00:00+00:00,56.766254,9.056604,2.1785,89.483276,0.6285,-1.960651,NaN,0.0,0.0,0.0,0.0,NaN,3.0,1033.300049,1030.100708,98.0,22.0,98.0,0.0,NaN,NaN,0.000930,0.075397,14.489720,NaN,NaN,NaN,63.435013,NaN,NaN,NaN,23.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-12-03 02:00:00+00:00,56.766254,9.056604,2.4285,84.478004,0.0785,-2.108335,NaN,0.0,0.0,0.0,0.0,NaN,3.0,1032.900024,1029.704834,93.0,1.0,93.0,0.0,NaN,NaN,0.008470,0.113278,16.622490,NaN,NaN,NaN,72.349792,NaN,NaN,NaN,27.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-03 03:00:00+00:00,56.766254,9.056604,1.8785,86.588997,-0.1215,-2.495600,NaN,0.0,0.0,0.0,0.0,NaN,3.0,1032.699951,1029.498901,85.0,85.0,5.0,0.0,NaN,NaN,0.004688,0.094109,15.294703,NaN,NaN,NaN,63.435013,NaN,NaN,NaN,27.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-12-03 04:00:00+00:00,56.766254,9.056604,2.0285,86.290878,-0.0215,-2.476788,NaN,0.0,0.0,0.0,0.0,NaN,3.0,1032.500000,1029.301270,97.0,97.0,0.0,0.0,NaN,NaN,0.005886,0.097237,16.299694,NaN,NaN,NaN,59.470375,NaN,NaN,NaN,26.639999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504595,2025-03-22 19:00:00+00:00,60.000000,8.979118,-3.0035,82.876480,-5.5035,-6.440490,NaN,0.0,0.0,0.0,0.0,NaN,0.0,1026.199951,932.741760,0.0,0.0,0.0,0.0,NaN,NaN,0.000000,0.084157,4.104631,NaN,NaN,NaN,74.744827,NaN,NaN,NaN,18.359999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
504596,2025-03-22 20:00:00+00:00,60.000000,8.979118,-3.7035,86.327667,-5.6535,-7.104300,NaN,0.0,0.0,0.0,0.0,NaN,0.0,1026.400024,932.694458,0.0,0.0,0.0,0.0,NaN,NaN,0.000000,0.063774,3.745557,NaN,NaN,NaN,54.782326,NaN,NaN,NaN,17.280001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
504597,2025-03-22 21:00:00+00:00,60.000000,8.979118,-4.0535,87.948997,-5.7535,-7.438463,NaN,0.0,0.0,0.0,0.0,NaN,0.0,1026.599976,932.760986,0.0,0.0,0.0,0.0,NaN,NaN,0.000000,0.054756,3.563818,NaN,NaN,NaN,44.999897,NaN,NaN,NaN,16.559999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
504598,2025-03-22 22:00:00+00:00,60.000000,8.979118,-4.5035,89.258003,-6.0035,-7.879004,NaN,0.0,0.0,0.0,0.0,NaN,0.0,1026.599976,932.612488,0.0,0.0,0.0,0.0,NaN,NaN,0.000000,0.047183,3.319036,NaN,NaN,NaN,40.601215,NaN,NaN,NaN,15.840000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504600 entries, 0 to 504599
Data columns (total 45 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   time                        504600 non-null  datetime64[ns, UTC]
 1   latitude                    504600 non-null  float64            
 2   longitude                   504600 non-null  float64            
 3   temperature_2m              504600 non-null  float32            
 4   relative_humidity_2m        504600 non-null  float32            
 5   dew_point_2m                504600 non-null  float32            
 6   apparent_temperature        504600 non-null  float32            
 7   precipitation_probability   0 non-null       float32            
 8   precipitation               504600 non-null  float32            
 9   rain                        504600 non-null  float32            
 10  showers                     504600 non-null 

In [ ]:
drop_cols = ['snow_depth']
df_weather.drop(columns=drop_cols, inplace=True)

In [ ]:
# Display dataframe with NaN Values
df_weather.isnull().sum()

time                               0
latitude                           0
longitude                          0
temperature_2m                     0
relative_humidity_2m               0
dew_point_2m                       0
apparent_temperature               0
precipitation_probability     504600
precipitation                      0
rain                               0
showers                            0
snowfall                           0
weather_code                       0
pressure_msl                       0
surface_pressure                   0
cloud_cover                        0
cloud_cover_low                    0
cloud_cover_mid                    0
cloud_cover_high                   0
visibility                    504600
evapotranspiration            504600
et0_fao_evapotranspiration         0
vapour_pressure_deficit            0
wind_speed_10m                     0
wind_speed_80m                504600
wind_speed_120m               504600
wind_speed_180m               504600
w

In [ ]:
df_weather_cleaned = df_weather.dropna(axis=1, how="all")
df_weather_cleaned = df_weather_cleaned.dropna(axis=0, how="any")
df_weather_cleaned = process_df(df_weather_cleaned)
df_weather_cleaned

,time,latitude,longitude,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,showers,snowfall,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_direction_10m,wind_gusts_10m
0,2022-12-03 00:00:00,56.766254,9.056604,1.8285,91.745644,0.6285,-2.225537,0.0,0.0,0.0,0.0,1.0,1033.500000,1030.295898,46.0,100.0,92.0,0.0,0.000000,0.057718,13.905509,68.749428,28.080000
1,2022-12-03 01:00:00,56.766254,9.056604,2.1785,89.483276,0.6285,-1.960651,0.0,0.0,0.0,0.0,3.0,1033.300049,1030.100708,98.0,22.0,98.0,0.0,0.000930,0.075397,14.489720,63.435013,23.400000
2,2022-12-03 02:00:00,56.766254,9.056604,2.4285,84.478004,0.0785,-2.108335,0.0,0.0,0.0,0.0,3.0,1032.900024,1029.704834,93.0,1.0,93.0,0.0,0.008470,0.113278,16.622490,72.349792,27.000000
3,2022-12-03 03:00:00,56.766254,9.056604,1.8785,86.588997,-0.1215,-2.495600,0.0,0.0,0.0,0.0,3.0,1032.699951,1029.498901,85.0,85.0,5.0,0.0,0.004688,0.094109,15.294703,63.435013,27.000000
4,2022-12-03 04:00:00,56.766254,9.056604,2.0285,86.290878,-0.0215,-2.476788,0.0,0.0,0.0,0.0,3.0,1032.500000,1029.301270,97.0,97.0,0.0,0.0,0.005886,0.097237,16.299694,59.470375,26.639999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504595,2025-03-22 19:00:00,60.000000,8.979118,-3.0035,82.876480,-5.5035,-6.440490,0.0,0.0,0.0,0.0,0.0,1026.199951,932.741760,0.0,0.0,0.0,0.0,0.000000,0.084157,4.104631,74.744827,18.359999
504596,2025-03-22 20:00:00,60.000000,8.979118,-3.7035,86.327667,-5.6535,-7.104300,0.0,0.0,0.0,0.0,0.0,1026.400024,932.694458,0.0,0.0,0.0,0.0,0.000000,0.063774,3.745557,54.782326,17.280001
504597,2025-03-22 21:00:00,60.000000,8.979118,-4.0535,87.948997,-5.7535,-7.438463,0.0,0.0,0.0,0.0,0.0,1026.599976,932.760986,0.0,0.0,0.0,0.0,0.000000,0.054756,3.563818,44.999897,16.559999
504598,2025-03-22 22:00:00,60.000000,8.979118,-4.5035,89.258003,-6.0035,-7.879004,0.0,0.0,0.0,0.0,0.0,1026.599976,932.612488,0.0,0.0,0.0,0.0,0.000000,0.047183,3.319036,40.601215,15.840000


In [ ]:
# Display dataframe with NaN Values
df_weather_cleaned.isnull().sum()

time                          0
latitude                      0
longitude                     0
temperature_2m                0
relative_humidity_2m          0
dew_point_2m                  0
apparent_temperature          0
precipitation                 0
rain                          0
showers                       0
snowfall                      0
weather_code                  0
pressure_msl                  0
surface_pressure              0
cloud_cover                   0
cloud_cover_low               0
cloud_cover_mid               0
cloud_cover_high              0
et0_fao_evapotranspiration    0
vapour_pressure_deficit       0
wind_speed_10m                0
wind_direction_10m            0
wind_gusts_10m                0
dtype: int64

In [ ]:
cords = set(zip(df_weather_cleaned['latitude'].astype(float), df_weather_cleaned['longitude'].astype(float)))
print(len(cords))
cords

25


{(53.53251266479492, 9.120458602905273),
 (53.53251266479492, 12.562141418457031),
 (53.53251266479492, 16.00382423400879),
 (53.53251266479492, 19.445505142211914),
 (53.53251266479492, 23.05927276611328),
 (55.149383544921875, 9.180000305175781),
 (55.149383544921875, 12.600000381469727),
 (55.149383544921875, 16.020000457763672),
 (55.149383544921875, 19.440000534057617),
 (55.149383544921875, 23.040000915527344),
 (56.76625442504883, 9.056604385375977),
 (56.76625442504883, 12.641510009765625),
 (56.76625442504883, 16.037736892700195),
 (56.76625442504883, 19.433961868286133),
 (56.76625442504883, 23.01886749267578),
 (58.38312530517578, 9.118942260742188),
 (58.38312530517578, 12.48898696899414),
 (58.38312530517578, 16.057268142700195),
 (58.38312530517578, 19.42731285095215),
 (58.38312530517578, 22.995594024658203),
 (60.0, 8.979118347167969),
 (60.0, 12.52900218963623),
 (60.0, 16.078886032104492),
 (60.0, 19.62877082824707),
 (60.0, 22.969837188720703)}

In [ ]:
df_weather_cleaned['latitude'].nunique(), df_weather_cleaned['longitude'].nunique()
#df_weather['time'].nunique()

(5, 25)

In [ ]:



#df_weather_cleaned.rename(columns={'latitude': 'latitude_weather', 'longitude': 'longitude_weather'}, inplace=True)
display(df_weather_cleaned.head())
df_weather_cleaned.info()

,time,latitude,longitude,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,showers,snowfall,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_direction_10m,wind_gusts_10m
0,2022-12-03 00:00:00,56.766254,9.056604,1.8285,91.745644,0.6285,-2.225537,0.0,0.0,0.0,0.0,1.0,1033.500000,1030.295898,46.0,100.0,92.0,0.0,0.000000,0.057718,13.905509,68.749428,28.080000
1,2022-12-03 01:00:00,56.766254,9.056604,2.1785,89.483276,0.6285,-1.960651,0.0,0.0,0.0,0.0,3.0,1033.300049,1030.100708,98.0,22.0,98.0,0.0,0.000930,0.075397,14.489720,63.435013,23.400000
2,2022-12-03 02:00:00,56.766254,9.056604,2.4285,84.478004,0.0785,-2.108335,0.0,0.0,0.0,0.0,3.0,1032.900024,1029.704834,93.0,1.0,93.0,0.0,0.008470,0.113278,16.622490,72.349792,27.000000
3,2022-12-03 03:00:00,56.766254,9.056604,1.8785,86.588997,-0.1215,-2.495600,0.0,0.0,0.0,0.0,3.0,1032.699951,1029.498901,85.0,85.0,5.0,0.0,0.004688,0.094109,15.294703,63.435013,27.000000
4,2022-12-03 04:00:00,56.766254,9.056604,2.0285,86.290878,-0.0215,-2.476788,0.0,0.0,0.0,0.0,3.0,1032.500000,1029.301270,97.0,97.0,0.0,0.0,0.005886,0.097237,16.299694,59.470375,26.639999


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504600 entries, 0 to 504599
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   time                        504600 non-null  datetime64[ns]
 1   latitude                    504600 non-null  float32       
 2   longitude                   504600 non-null  float32       
 3   temperature_2m              504600 non-null  float32       
 4   relative_humidity_2m        504600 non-null  float32       
 5   dew_point_2m                504600 non-null  float32       
 6   apparent_temperature        504600 non-null  float32       
 7   precipitation               504600 non-null  float32       
 8   rain                        504600 non-null  float32       
 9   showers                     504600 non-null  float32       
 10  snowfall                    504600 non-null  float32       
 11  weather_code                504600 non-

## Plot Ocean and Weather Data Coordinates and Target

In [ ]:
import folium

df_map = df_weather_cleaned.groupby(by=["latitude", "longitude"]).mean().reset_index()
# Create a folium map centered around the average lat/lon
map_center = [df_map["latitude"].mean(), df_map["longitude"].mean()]
m = folium.Map(location=map_center, zoom_start=5)

# Add markers
for _, row in df_map.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        #popup=f"Time: {row['time']}<br>Depth: {row['depth']}<br>BottomT: {row['bottomT']}",
        tooltip=f"Lat: {row['latitude']}, Lon: {row['longitude']}",
        icon=folium.Icon(color="blue")
    ).add_to(m)


m

In [ ]:
# Zeitdifferenzen berechnen
time_diff = df_weather_cleaned["time"].diff()

# Falls der Zeitabstand konstant sein sollte (z. B. 1 Stunde), prüfe Abweichungen:
expected_interval = pd.Timedelta(hours=1)  # Anpassen je nach erwartetem Intervall
missing_times = df_weather_cleaned["time"][time_diff > expected_interval]

# Fehlende Zeitpunkte ausgeben
if missing_times.empty:
    print("Keine fehlenden Zeitpunkte!")
else:
    print("Fehlende Zeitpunkte erkannt:")
    print(missing_times)

Keine fehlenden Zeitpunkte!


In [ ]:
numerical_data_path = Path(f'../data/numerical_data/points{points}')

In [ ]:
# Save Data as json and csv
file_path = Path(numerical_data_path, f'df_weather{points}.json')
df_weather_cleaned.to_json(file_path, orient='records', lines=True, date_format='iso', date_unit='s')

file_path = Path(numerical_data_path, f'df_weather{points}.csv')
df_weather_cleaned.to_csv(file_path, index=False)




In [ ]:
# DataFrame in xarray Dataset umwandeln
ds = df_weather_cleaned.set_index(["time", "latitude", "longitude"]).to_xarray()

ds

<xarray.Dataset> Size: 202MB
Dimensions:                     (time: 20184, latitude: 5, longitude: 25)
Coordinates:
  * time                        (time) datetime64[ns] 161kB 2022-12-03 ... 20...
  * latitude                    (latitude) float32 20B 53.53 55.15 ... 60.0
  * longitude                   (longitude) float32 100B 8.979 9.057 ... 23.06
Data variables: (12/20)
    temperature_2m              (time, latitude, longitude) float32 10MB nan ...
    relative_humidity_2m        (time, latitude, longitude) float32 10MB nan ...
    dew_point_2m                (time, latitude, longitude) float32 10MB nan ...
    apparent_temperature        (time, latitude, longitude) float32 10MB nan ...
    precipitation               (time, latitude, longitude) float32 10MB nan ...
    rain                        (time, latitude, longitude) float32 10MB nan ...
    ...                          ...
    cloud_cover_high            (time, latitude, longitude) float32 10MB nan ...
    et0_fao_evapotranspiration  (time, latitude, longitude) float32 10MB nan ...
    vapour_pressure_deficit     (time, latitude, longitude) float32 10MB nan ...
    wind_speed_10m              (time, latitude, longitude) float32 10MB nan ...
    wind_direction_10m          (time, latitude, longitude) float32 10MB nan ...
    wind_gusts_10m              (time, latitude, longitude) float32 10MB nan ...

In [ ]:
file_path = Path(numerical_data_path, f'df_weather{points}.nc')

# Als .nc-Datei speichern
ds.to_netcdf(file_path)